## 1 Overview
An enviroment to train and evaluate neural networks on learning logical consequence.

In [ ]:
# For Google Collab: Get repository and go to it in collab.
!git clone https://github.com/stereifberger/master-s-thesis-finished
%cd /content/master-s-thesis-finished/

In [ ]:
# For VsCode after starting Jupyter server: go to right directory.
%cd master-s-thesis-finished/

In [ ]:
# Install required dependencies - not necessary on google colab
!pip install -r requirements.txt

In [ ]:
# Import required libraries
from imports import *

## 2 Create dataset
First the dataset for training is generated. For this the function "create_dataset" from "generation.py" utilizes the functions "get_conclusions" to generate a set of random starting formulas, for which iterativly the applicability of rules is checked. All applicable rules are then used to generate new derivations. In each iteration of get_conclusions, set by the iterations variable, new, longer examples are generated.

**Rules.** The rules are defined in calculi.py. Two sets are avaiable: Intuitionistic propositional logic (set below via "calculus = ipl") and classical propositional logic (set below via "calculus = cl").

**Dataset entries.**
- **x_train.** Training input: [INDEX, PREMISES,CONCLUSION]
- **y_train_ordered.** Dataset of correct derivations where each sublist i correspnds to INDEX: [DERIVATIONS_0...DERIVATION_N]

**Encoding.** Propositional variables and logical constants are encoded as integers.

**Example entries withouth numerical representation and one-hot-encoding.**
- **x_train.** [2345, A, A THEN B, DERIVES, B OR C]
- **y_train_ordered.** Sublist 2345 is entry entry: [[A, A THEN B, B, B OR C], [A, A THEN B, B, A AND B, B OR C]]


In [ ]:
# Create dataset
x_train_2d, x_train_3d, y_train_ordered, max_y_train_len = generation.create_dataset(iterations = [1,2], calculus = calculi.cl)

In [ ]:
import json
torch.save(x_train_2d, 'x_train_2d.pt')
torch.save(x_train_3d, 'x_train_3d.pt')
torch.save(y_train_ordered, 'y_train_ordered.pt')
with open('Medium_max_y_train_len.json', 'w') as file:
    json.dump(max_y_train_len, file)

## 3 Prepare dataset and define models for training
Next with pytorch's dataloader the single training entries in x_train are assigned to batches of size "batch size" in mixed order. Then the different models are defined using definitions from "architectures.py". These models are:

- Feedforward network (net)
- Recurrent neural network (RNNNet)
- Long-short-term memory (LSTMNet)
- Transformers (TransformerModel)

In [ ]:
# Use when gpu is present to empty its catch and define it as "device" for referencing it
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Get the datasets' shapes for the model definitions later
two_d_shape = x_train_2d.shape
three_d_shape = x_train_3d.shape
max_y_length = int(max_y_train_len/14)

In [ ]:
# Reverse one-hot encoding for encoder-decoder models
x = torch.argmax(x_train_3d, dim=2)
x[:, 0] = x_train_2d[:, 0]
x_train_nu = x

In [ ]:
# Set train-test split to 80-20 [^1]
train_size = int(0.8 * len(x_train_2d))
test_size = len(x_train_2d) - train_size
x_train_2d, x_test_2d = random_split(x_train_2d, [train_size, test_size])
x_train_3d, x_test_3d = random_split(x_train_3d, [train_size, test_size])
x_train_nu, x_test_nu = random_split(x_train_nu, [train_size, test_size])

In [ ]:
# Collect and mix the data in [^2]
train_dataloader_2d = DataLoader(dataset = x_train_2d, shuffle = True, batch_size = 50)
test_dataloader_2d = DataLoader(dataset = x_test_2d, shuffle = True, batch_size = 50)
train_dataloader_3d = DataLoader(dataset = x_train_3d, shuffle = True, batch_size = 50)
test_dataloader_3d = DataLoader(dataset = x_test_3d, shuffle = True, batch_size = 50)
train_dataloader_nu = DataLoader(dataset = x_train_nu, shuffle = True, batch_size = 50)
test_dataloader_nu = DataLoader(dataset = x_test_nu, shuffle = True, batch_size = 50)

In [ ]:
# Load ground truth data to GPU
y_train = y_train_ordered.to(device)
y_train_3d = y_train.view(int(len(y_train)), int(len(y_train[0])), int(len(y_train[0][0])/14), 14)

In [ ]:
# Define the Encoder-Decoder networks
## FFN | Inputs: input_dim, hidden dim
encoder_ffn = architectures.Encoder_FFN(three_d_shape[1], 150)
decoder_ffn = architectures.Decoder_FFN((max_y_length*14), 150)
ffn_ed_model = architectures.Seq2Seq(encoder_ffn, decoder_ffn, device)
## LSTM | Inputs: input_dim, embedding dim, hidden dim, nr layers, droput
encoder_lstm = architectures.Encoder_LSTM(three_d_shape[1], 150, 150, 2, 0.5)
decoder_lstm = architectures.Decoder_LSTM(14, 150, 150, 3, 0.5)
lst_ed_model = architectures.Seq2Seq(encoder_lstm, decoder_lstm, device)
## Transformer | Inputs: input_dim, embedding dim, hidden dim, nr layers, droput
encoder_tra = architectures.TransformerEncoder(three_d_shape[1], 150, 5, 150, 1, dropout=0.1)
decoder_tra = architectures.TransformerDecoder(14, 150, 1, 150, 3)
tra_ed_model = architectures.Seq2SeqTransformer(encoder_tra, decoder_tra, device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
# Define optimizers for models
lr = 0.001
ffn_ed_optimizer = torch.optim.Adam(ffn_ed_model.parameters(),lr=lr)
lst_ed_optimizer = torch.optim.Adam(lst_ed_model.parameters(),lr=lr)
tra_ed_optimizer = torch.optim.Adam(tra_ed_model.parameters(),lr=lr)

# 4 Training

In [ ]:
criterion = nn.CrossEntropyLoss()

## 4.1 Encoder-Decoder Networks

### 4.1.1 FFN Encoder-Decoder

In [ ]:
# Load model to GPU
ffn_ed_model.to(device)

In [ ]:
# Train model and save results
FFN_CELtrain, FFN_CELtest, FFN_ACCtrain, FFN_ACCtest = schedule.train_model(ffn_ed_model, train_dataloader_nu, test_dataloader_nu, ffn_ed_optimizer, criterion, 50, device, max_y_length, y_train)
torch.save(ffn_ed_model.state_dict(), 'addition_model.pth')

In [ ]:
# A sanity test for wheter the outputs look appropriate
schedule.sanity(ffn_ed_model, test_dataloader_nu, device, max_y_length)

In [ ]:
# Delete model from GPU to make space for new models
del ffn_ed_model
torch.cuda.empty_cache()

### 4.1.3 LSTM Encoder-Decoder

In [ ]:
# Load model to GPU
lst_ed_model.to(device)

In [ ]:
# Training Loop
LSTM_CELtrain, LSTM_CELtest, LSTM_ACCtrain, LSTM_ACCtest = schedule.train_model(lst_ed_model, train_dataloader_nu, test_dataloader_nu, lst_ed_optimizer, criterion, 50, device, max_y_length, y_train_3d)
torch.save(lst_ed_model.state_dict(), 'addition_model.pth')

In [ ]:
# A sanity test for wheter the outputs look appropriate
schedule.sanity(lst_ed_model, test_dataloader_nu, device, max_y_length)

In [ ]:
# Delete model from GPU to make space for new models
del lst_ed_model
torch.cuda.empty_cache()

### 4.1.4 Transformer

In [ ]:
importlib.reload(architectures)

<module 'architectures' from '/content/master-s-thesis-finished/architectures.py'>

In [ ]:
# Load model to GPU
tra_ed_model.to(device)

In [ ]:
TRA_CELtrain, TRA_CELtest, TRA_ACCtrain, TRA_ACCtest = schedule.train_model(tra_ed_model, train_dataloader_nu, test_dataloader_nu, tra_ed_optimizer, criterion, 50, device, max_y_length, y_train_3d)
torch.save(tra_ed_model.state_dict(), 'addition_model.pth')

Ep. 01, CEL-Train: 0.9663| CEL-Test: 0.5746 | ACC-Train: 0.8750 | ACC-Test:  0.8239


In [ ]:
# A sanity test for wheter the outputs look appropriate
schedule.sanity(tra_ed_model, test_dataloader_nu, device, max_y_length)

In [ ]:
# Delete model from GPU to make space for new models
del tra_ed_model
torch.cuda.empty_cache()